In [ ]:
# /home/labuser/Documents/Level2_Day1_Data/supply_chain_orders.csv

In [3]:
#Importing Libs
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, rank, dense_rank, sum, avg, lag, lead

In [4]:
# Initialize Spark
spark = SparkSession.builder.appName("SupplyChainAnalysis").getOrCreate()

In [9]:
# Load Data from CSV file
df = spark.read.option("header", "true").csv("/home/labuser/Documents/Level2_Day1_Data/supply_chain_orders.csv", inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- order_id: string (nullable = true)
 |-- warehouse: string (nullable = true)
 |-- supplier: string (nullable = true)
 |-- product: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- delivery_time: integer (nullable = true)

+--------+---------+----------+----------+--------+----------+-------------+
|order_id|warehouse|  supplier|   product|quantity|order_date|delivery_time|
+--------+---------+----------+----------+--------+----------+-------------+
|   ORD_1|     WH_B|Supplier_4|    Laptop|      90|2024-01-30|            5|
|   ORD_2|     WH_C|Supplier_4|    Laptop|      96|2024-01-08|            6|
|   ORD_3|     WH_B|Supplier_1|Headphones|      29|2024-01-02|            1|
|   ORD_4|     WH_B|Supplier_4|    Laptop|      97|2024-01-03|            1|
|   ORD_5|     WH_B|Supplier_1|    Tablet|      39|2024-01-30|            2|
+--------+---------+----------+----------+--------+----------+-------------+
only showing

In [10]:
# Find Top 3 Suppliers per Warehouses
window_spec = Window.partitionBy("warehouse").orderBy(col("quantity").desc())
df_ranked = df.withColumn("rank", rank().over(window_spec))
df_ranked.filter(col("rank")<= 3).show()

+--------+---------+----------+----------+--------+----------+-------------+----+
|order_id|warehouse|  supplier|   product|quantity|order_date|delivery_time|rank|
+--------+---------+----------+----------+--------+----------+-------------+----+
| ORD_156|     WH_A|Supplier_2|    Tablet|      98|2024-01-17|            1|   1|
|  ORD_59|     WH_A|Supplier_1|Smartphone|      97|2024-01-29|            5|   2|
|  ORD_41|     WH_A|Supplier_3|Headphones|      95|2024-01-26|            6|   3|
|  ORD_53|     WH_B|Supplier_3|        TV|     100|2024-01-15|            2|   1|
|   ORD_9|     WH_B|Supplier_4|Headphones|      98|2024-01-17|            2|   2|
|   ORD_4|     WH_B|Supplier_4|    Laptop|      97|2024-01-03|            1|   3|
|  ORD_10|     WH_C|Supplier_3|    Laptop|      99|2024-01-13|            2|   1|
| ORD_113|     WH_C|Supplier_4|Headphones|      98|2024-01-03|            7|   2|
| ORD_178|     WH_C|Supplier_2|    Tablet|      98|2024-01-19|            1|   2|
+--------+------

In [11]:
# Calculated cumulative order quantity per supplier
    window_spec2 = Window.partitionBy("supplier").orderBy("order_date").rowsBetween(Window.unboundedPreceding, Window.currentRow)
df_cumulative = df.withColumn("cumulative_quantity", sum("quantity").over(window_spec2))
df_cumulative.show()

+--------+---------+----------+----------+--------+----------+-------------+-------------------+
|order_id|warehouse|  supplier|   product|quantity|order_date|delivery_time|cumulative_quantity|
+--------+---------+----------+----------+--------+----------+-------------+-------------------+
|  ORD_52|     WH_C|Supplier_1|Headphones|      57|2024-01-01|            1|                 57|
| ORD_196|     WH_A|Supplier_1|        TV|      29|2024-01-01|            4|                 86|
|   ORD_3|     WH_B|Supplier_1|Headphones|      29|2024-01-02|            1|                115|
|  ORD_38|     WH_B|Supplier_1|        TV|       1|2024-01-02|            7|                116|
|  ORD_58|     WH_B|Supplier_1|    Laptop|      39|2024-01-02|            2|                155|
| ORD_180|     WH_B|Supplier_1|        TV|      39|2024-01-02|            6|                194|
| ORD_192|     WH_B|Supplier_1|        TV|      12|2024-01-02|            7|                206|
|  ORD_29|     WH_B|Supplier_1

In [12]:
# Compare current vs previous delivery times
window_spec3 = Window.partitionBy("supplier").orderBy("order_date")

df_lead_lag = df.withColumn("previous_delivery", lag("delivery_time", 1).over(window_spec3)).withColumn("next_delivery", lead("delivery_time", 1).over(window_spec3))
df_lead_lag.show()


+--------+---------+----------+----------+--------+----------+-------------+-----------------+-------------+
|order_id|warehouse|  supplier|   product|quantity|order_date|delivery_time|previous_delivery|next_delivery|
+--------+---------+----------+----------+--------+----------+-------------+-----------------+-------------+
|  ORD_52|     WH_C|Supplier_1|Headphones|      57|2024-01-01|            1|             null|            4|
| ORD_196|     WH_A|Supplier_1|        TV|      29|2024-01-01|            4|                1|            1|
|   ORD_3|     WH_B|Supplier_1|Headphones|      29|2024-01-02|            1|                4|            7|
|  ORD_38|     WH_B|Supplier_1|        TV|       1|2024-01-02|            7|                1|            2|
|  ORD_58|     WH_B|Supplier_1|    Laptop|      39|2024-01-02|            2|                7|            6|
| ORD_180|     WH_B|Supplier_1|        TV|      39|2024-01-02|            6|                2|            7|
| ORD_192|     WH_B

In [13]:
# Calculate a 3 days moving average for delivery time
window_spec4 = Window.partitionBy("supplier").orderBy("order_date").rowsBetween(-2, 0) # 3 rows at a time (current row + 2 previous row)
df_moving_avg = df.withColumn("avg_delivery_time",avg("delivery_time").over(window_spec4))
df_moving_avg.show()

+--------+---------+----------+----------+--------+----------+-------------+------------------+
|order_id|warehouse|  supplier|   product|quantity|order_date|delivery_time| avg_delivery_time|
+--------+---------+----------+----------+--------+----------+-------------+------------------+
|  ORD_52|     WH_C|Supplier_1|Headphones|      57|2024-01-01|            1|               1.0|
| ORD_196|     WH_A|Supplier_1|        TV|      29|2024-01-01|            4|               2.5|
|   ORD_3|     WH_B|Supplier_1|Headphones|      29|2024-01-02|            1|               2.0|
|  ORD_38|     WH_B|Supplier_1|        TV|       1|2024-01-02|            7|               4.0|
|  ORD_58|     WH_B|Supplier_1|    Laptop|      39|2024-01-02|            2|3.3333333333333335|
| ORD_180|     WH_B|Supplier_1|        TV|      39|2024-01-02|            6|               5.0|
| ORD_192|     WH_B|Supplier_1|        TV|      12|2024-01-02|            7|               5.0|
|  ORD_29|     WH_B|Supplier_1|        T